In [22]:
import cv2
import tkinter as tk
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import os
import random

img_counter = 0
frame_counter = 0


#downscale and upscale image from path
def preprocess(path,scale):
    img = cv2.imread(path)
    img = cv2.resize(img, (int(img.shape[1]/scale), int(img.shape[0]/scale)))
    img = cv2.resize(img, (int(img.shape[1]*scale), int(img.shape[0]*scale)))
    
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    
    return img

#slice image into patches of size 33x33 with stride 14
def slice_image(img):
    patches = []
    global frame_counter
    counter = 0
    for i in range(0,img.shape[0]-32,14):
        for j in range(0,img.shape[1]-32,14):
            counter += 1
            if counter < frame_counter:
                break
            frame_counter += 1
            patches.append(img[i:i+33,j:j+33,:])
    return patches

def get_images(dirpath,num_images):
    images = []
    images_orig = []
    global img_counter
    counter = 0
    while True:
        for file in os.listdir(dirpath):
            counter += 1
            if counter < img_counter:
                break
            img_counter += 1
            img_orig = preprocess(dirpath+file,1)
            img = preprocess(dirpath+file,random.randint(2,4))
            patches = slice_image(img)
            patches_orig = slice_image(img_orig)
            for p in range(len(patches)):
                images.append(patches[p])
                images_orig.append(patches_orig[p])
                if len(images) == num_images:
                    return images,images_orig

In [23]:
class SuperRes(nn.Module):
    def __init__(self):
        super(SuperRes, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x1 = x[:,:,0]
        x1 = self.relu(self.conv1(x1))
        x1 = self.relu(self.conv2(x1))
        x1 = self.conv3(x1)
        return x1
    
def train(num_images,model, optimizer, loss_fn, epochs, print_step):
    for epoch in range(epochs):
        counter = 0
        loss_sum = 0
        for img,img_orig in get_images('T91/train/',num_images):
            counter += 1
            optimizer.zero_grad()
            output = model(img)
            loss = loss_fn(output, img_orig)
            loss.backward()
            optimizer.step()
            loss_sum += loss.item()
            if counter % print_step == 0:
                print('Epoch %d, Batch %d loss: %.6f' % (epoch, counter, loss.item()))
                loss_sum = 0

In [24]:
model = SuperRes()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

loss_fn = nn.MSELoss()

epochs = 5000

num_images = 32

train(num_images,model, optimizer, loss_fn, epochs, 10)

IndexError: list index out of range